# Determining a Location for a Newfoundland Fish & Chips Shop

Newfoundland fish and chips are a favourite where I am from.  Newfoundlanders, unable to find themselves jobs at home, have left the island to work in other provinces, many finding themselves in northern Alberta and southern Ontario.  This will be my attempt at finding an ideal location, in Toronto, for my Newfoundland fish and chips shop to bring old-fashioned, East-Coast cooking to those living so far from home.

In [1]:
# As always, I loaded the libraries I will need to run the code.

import pandas as pd # I imported pandas and set my display limits as before.
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

from pandas.io.json import json_normalize # I need this to work with JSON and convert the data to a dataframe.

import numpy as np # I imporeted NumPy.

import json # I imported json.

from geopy.geocoders import Nominatim # I imported geopy.

import requests # I imported requests for the data to be drawn from Foursquare.

print('Done!') # I have had issues with Watson Studio lately not loading the environment properly.  I added this line to let me know if and when the process is complete.

Done!


## Competition

The first step will be to find out if there is another business in Toronto boasting Newfoundland cooking.  This of course, will be done by drawing data from Foursquare.  A get command will be sent to the Foursquare database with Newfoundland as the venue identifier.

In [2]:
# I fed my Foursquare credentials into Python.

CLIENT_ID = 'L3FOESFPXZSNGOVOCS0XKQXA0BHFNBLK3R2EMUZMKFJ1X4GF' # My client ID.

CLIENT_SECRET = 'IDR0V0BKV2IEIXVCIVT54J0N2OVLNZ3M31IRE5A2V4JE3YPP' # My client secret.

VERSION = '20180605'

In [3]:
# I fond the longitude and latitude coordinates for Toronto.

address = 'Toronto, ON'
geolocator = Nominatim(user_agent='ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('Toronto is at {}, {}.'.format(latitude, longitude))

Toronto is at 43.6534817, -79.3839347.


In [4]:
# I set my desired parameters with "Newfounlad" as my search query.  I gave myself enough room to search, setting the radius to 5km and set the result limit to 50.

search_query = 'Newfoundland'
radius = 5000
LIMIT = 50
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=L3FOESFPXZSNGOVOCS0XKQXA0BHFNBLK3R2EMUZMKFJ1X4GF&client_secret=IDR0V0BKV2IEIXVCIVT54J0N2OVLNZ3M31IRE5A2V4JE3YPP&ll=43.6534817,-79.3839347&v=20180605&query=Newfoundland&radius=5000&limit=50'

In [5]:
# I requested the result in JSON form.

competition_results = requests.get(url).json()

In [6]:
# I assigned what was relevant to the identifier 'competition'.

competition = competition_results['response']['venues']

In [7]:
# I transformed competition into a dataframe.

competition_df = json_normalize(competition)
competition_df.head()

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.postalCode,location.state,name,referralId
0,"[{'id': '4bf58dd8d48988d1f9931735', 'name': 'R...",False,4c9136fe51d9b1f760a27e46,Lake Shore,CA,Toronto,Canada,Newfoundland,3192,"[Lake Shore (Newfoundland), Toronto ON, Canada]","[{'label': 'display', 'lat': 43.632488, 'lng':...",43.632488,-79.410939,NaN,ON,Lake Shore & Newfoundland rd,v-1592781374
1,"[{'id': '4bf58dd8d48988d128951735', 'name': 'G...",False,5cd7262131ac6c002ca05388,1537 Queen St W,CA,Toronto,Canada,NaN,4998,"[1537 Queen St W, Toronto ON M6R 1A7, Canada]","[{'label': 'display', 'lat': 43.639311, 'lng':...",43.639311,-79.442812,M6R 1A7,ON,The Newfoundland Store,v-1592781374


In [8]:
# I filtered out the unnecessary data from the dataframe.

filtered_competition = ['name', 'categories', 'location.lat', 'location.lng', 'location.postalCode']
competition_filtered = competition_df.loc[:, filtered_competition]

In [9]:
# I extracted the category of the venue.

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [10]:
# I filtered the category for each row.

competition_filtered['categories'] = competition_filtered.apply(get_category_type,axis=1)

In [11]:
# Cleaned up the data.

competition_filtered.columns = [column.split('.')[-1] for column in competition_filtered.columns]
competition_filtered

,name,categories,lat,lng,postalCode
0,Lake Shore & Newfoundland rd,Road,43.632488,-79.410939,NaN
1,The Newfoundland Store,Gift Shop,43.639311,-79.442812,M6R 1A7


## Review of the Results

From the results, I saw that two entries were returned; one entry was the name of a road in Toronto, "Newfoundland Road", while the other was, happily, a Newfoundland gift shop in the city.  There was, therefore, no competition for my chip shop.  So, I decided to check Foursquare for "fish & chips".

In [12]:
# I ran the 'fish & chips' search.

search_query2 = 'fish & chips'
radius = 5000
LIMIT = 50
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET,latitude, longitude, VERSION, search_query2, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=L3FOESFPXZSNGOVOCS0XKQXA0BHFNBLK3R2EMUZMKFJ1X4GF&client_secret=IDR0V0BKV2IEIXVCIVT54J0N2OVLNZ3M31IRE5A2V4JE3YPP&ll=43.6534817,-79.3839347&v=20180605&query=fish & chips&radius=5000&limit=50'

In [13]:
# I processed the data the same as before.

fish_results = requests.get(url).json()

fish_competition = fish_results['response']['venues']

fish_df = json_normalize(fish_competition)

filtered_fish = ['name', 'categories', 'location.lat', 'location.lng', 'location.postalCode']

fishy_competition = fish_df.loc[:, filtered_fish]

def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']
    
fishy_competition['categories'] = fishy_competition.apply(get_category_type,axis=1)

fishy_competition.columns = [column.split('.')[-1] for column in fishy_competition.columns]

print('This is what I found:')

fishy_competition.head(10)

This is what I found:


,name,categories,lat,lng,postalCode
0,London Style Fish And Chips,Food Court,43.655236,-79.381628,M5H 3S4
1,St. Lawrence Fish Market,Fish Market,43.648490,-79.371470,NaN
2,Captain Cookies Fish & Chips,Fish & Chips Shop,43.656492,-79.365291,NaN
3,The Battered Fish,Fish & Chips Shop,43.650196,-79.389754,NaN
4,Fish Noodle,Noodle House,43.659476,-79.393680,M5T 1L1
5,Coral Sea Fish Market,Fish Market,43.654620,-79.401517,NaN
6,Fresco's Fish & Chips,Fish & Chips Shop,43.654145,-79.401803,M5T 2L4
7,Square Fish,Seafood Restaurant,43.648451,-79.397267,M5V 2A9
8,Asiya Fish and Chips,Fish & Chips Shop,43.664519,-79.380276,M4Y
9,Time Square Fish & Chips,Fish & Chips Shop,43.648269,-79.370274,NaN


In [14]:
# I dropped all results with categories other than 'Fish & Chips Shop'.

chips_shops = fishy_competition[fishy_competition.categories == 'Fish & Chips Shop']
chips_shops

,name,categories,lat,lng,postalCode
2,Captain Cookies Fish & Chips,Fish & Chips Shop,43.656492,-79.365291,NaN
3,The Battered Fish,Fish & Chips Shop,43.650196,-79.389754,NaN
6,Fresco's Fish & Chips,Fish & Chips Shop,43.654145,-79.401803,M5T 2L4
8,Asiya Fish and Chips,Fish & Chips Shop,43.664519,-79.380276,M4Y
9,Time Square Fish & Chips,Fish & Chips Shop,43.648269,-79.370274,NaN
13,Beer Batter Fish & Chips,Fish & Chips Shop,43.663846,-79.370442,M5A 2L1
19,Harbord Fish and Chips,Fish & Chips Shop,43.662197,-79.406512,NaN
27,Reliable fish and chips,Fish & Chips Shop,43.658400,-79.351602,NaN
31,Sea Witch Fish and Chips,Fish & Chips Shop,43.682112,-79.423975,M6C 1A9
35,john & sons fish and chips shop,Fish & Chips Shop,43.685818,-79.393126,NaN


## Incorporating Population Data for Tornoto.

I retrieved Toronto postal code coordinates from a previous Coursera lab.

In [15]:
# Using data provided by a previous Coursera lab, I added the coordinates of the postal codes to the dataframe.

coordinates_data = pd.read_csv("http://cocl.us/Geospatial_data/Geospacial_coordinates.csv")
coordinates_df = coordinates_data
coordinates_df.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Population data, by postal code as it happens, is provided by StatCan on their website.

In [16]:
# I pulled the data from the website and created a dataframe.

population_by_code = pd.read_html("https://www12.statcan.gc.ca/census-recensement/2016/dp-pd/hlt-fst/pd-pl/Table.cfm?Lang=Eng&T=1201&SR=1&S=22&O=A&RPP=9999&PR=0")
population_df = population_by_code[0]
population_df.head()

,Geographic name,"Population, 2016","Total private dwellings, 2016","Private dwellings occupied by usual residents, 2016"
0,NaN,NaN,NaN,NaN
1,CanadaFootnote 1,35151728.0,15412443.0,14072079.0
2,A0A,46587.0,26155.0,19426.0
3,A0B,19792.0,13658.0,8792.0
4,A0C,12587.0,8010.0,5606.0


In [17]:
# I dropped all unnecessary postal codes, because the website accounted for all of Canada.

population = population_df[:-645]
population = population_df.drop(population_df.index[0:896])

In [18]:
# Again, I had to drop all unnecessary columns from the new dataframe and normalize it to the previous one.

population = population.drop(['Total private dwellings, 2016', 'Private dwellings occupied by usual residents, 2016'], axis=1)
population = population.rename(columns={'Geographic name': 'Postal Code'})
population.head()

,Postal Code,"Population, 2016"
896,M1B,66108.0
897,M1C,35626.0
898,M1E,46943.0
899,M1G,29690.0
900,M1H,24383.0


In [19]:
# I merged the two dataframes together.

pop_coords = pd.merge(population, coordinates_df, on='Postal Code')
pop_coords.head()

,Postal Code,"Population, 2016",Latitude,Longitude
0,M1B,66108.0,43.806686,-79.194353
1,M1C,35626.0,43.784535,-79.160497
2,M1E,46943.0,43.763573,-79.188711
3,M1G,29690.0,43.770992,-79.216917
4,M1H,24383.0,43.773136,-79.239476


## Determining the Best Location

Finally, I determined what the best location for my chip shop.  I did this by determining the population centre of the population of Toronto, assuming a uniform distribution of Newfoundlanders througout the city.

In [20]:
# To run the calculation, I had to transform the dataframe into a NumPy array.

pop_array = pop_coords.values

In [21]:
# I then calculated the population centroid.

pop_centroid = np.mean(pop_array[:,-3:], axis=0)
pop_centroid

array([26785.676470588234, 43.70527089215686, -79.39500912647058],
      dtype=object)

The population centre of Toronto is located at latitude 43.70, longitude -79.39, a few blocks away from The Newfoundland Store - the previously found gift shop - at latitude 43.64, longitude -79.44.  If I were to choose a location further south, I would bring my business closer to tourist attractions, including the store.  However, as I am catering to the Newfoundlanders who find themselves forced away from home, chasing work, I would select the centre of the population, providing a vast majority the easiest possible access.